In [1]:
import pandas as pd

In [2]:
df= pd.read_csv(r"C:\Users\tanay\Desktop\ATS\datasets\jobs_dataset_with_features.csv")

In [3]:
df.shape

(1615940, 2)

In [4]:
df.head()

,Role,Features
0,Social Media Manager,5 to 15 Years Digital Marketing Specialist M.T...
1,Frontend Web Developer,"2 to 12 Years Web Developer BCA HTML, CSS, Jav..."
2,Quality Control Manager,0 to 12 Years Operations Manager PhD Quality c...
3,Wireless Network Engineer,4 to 11 Years Network Engineer PhD Wireless ne...
4,Conference Manager,1 to 12 Years Event Manager MBA Event planning...


In [5]:
len(df["Role"].value_counts())

376

In [6]:
#dropping classes with less than 6500 instances
min_count=6500
role_counts=df['Role'].value_counts()
dropped_classes=role_counts[role_counts<min_count].index
filtered_df=df[~df['Role'].isin(dropped_classes)].reset_index(drop=True)

#checking the updated role counts
filtered_df['Role'].value_counts()

Interaction Designer          20580
Network Administrator         17470
User Interface Designer       14036
Social Media Manager          13945
User Experience Designer      13935
                              ...  
Benefits Coordinator           6839
Research Analyst               6830
Administrative Coordinator     6803
IT Support Specialist          6799
UI/UX Designer                 6743
Name: Role, Length: 61, dtype: int64

In [7]:
len(filtered_df['Role'].value_counts())

61

In [8]:
df=filtered_df.sample(n=10000)

In [9]:
df.head()

,Role,Features
184901,Data Analyst,0 to 15 Years Marketing Analyst BCA Data analy...
6331,Procurement Analyst,3 to 14 Years Procurement Manager M.Tech Procu...
366183,Office Manager,5 to 9 Years Executive Assistant M.Tech Office...
404323,Procurement Analyst,4 to 10 Years Procurement Manager MBA Procurem...
432458,Social Media Manager,3 to 13 Years Digital Marketing Specialist B.C...


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

x=df['Features']
y=df['Role']

#train-test-split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

#TF-IDF vectorization
tfidf_vectorizer=TfidfVectorizer()
x_train_tfidf=tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf=tfidf_vectorizer.transform(x_test)


In [11]:
#random forest classifier
rf_classifier=RandomForestClassifier()
rf_classifier.fit(x_train_tfidf,y_train)

#predictions
y_pred=rf_classifier.predict(x_test_tfidf)
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy",accuracy)

Accuracy 1.0


In [12]:
#clean resume
import re
def cleanResume(txt):
    cleanText=re.sub('http\S+\s',' ',txt)
    cleanText=re.sub('RT|cc','.',cleanText)
    cleanText=re.sub('#\S+\s','.',cleanText)
    cleanText=re.sub('@\S+',' ',cleanText)
    cleanText=re.sub('[%s]'%re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""),'',cleanText)
    cleanText=re.sub(r'[^\x00-\x7f]', ' ', cleanText) 
    cleanText=re.sub('\s+', ' ', cleanText)
    return cleanText

#prediction and category name
def job_recommendation(resume_text):
    resume_text=cleanResume(resume_text)
    resume_tfidf=tfidf_vectorizer.transform([resume_text])
    predicted_category=rf_classifier.predict(resume_tfidf)[0]
    return predicted_category

In [13]:
#EXAMPLE USAGE
resume_file="""VIT Bhopal University
Sep 2021 – Ongoing
Bachelor of Technology in Computer Science
Specialization: Artificial Intelligence and Machine Learning
CGPA: 8.31
Glendale Academy International, Hyderabad
June 2021
Senior Secondary Certificate Examination (Class XII)
Percentage: 81 %
Bharatiya Vidya Bhavans Vidyashram, Hyderabad
June 2019
Secondary Certificate Examination (Class X)
Percentage: 86 %
Academic Projects
AI-Cop Python, Keras, TensorFlow, CNN, Pandas, Jupyter Jul 22 - Oct 22
• Developed an AI model utilizing deep learning techniques to classify chest X-ray images into three distinct categories:
Covid-19 pneumonia, non-Covid-19 pneumonia, and healthy (negative)
• Applied image processing and convolutional neural networks (CNN) to enhance diagnostic accuracy
• Team Project: Collaborated with a team of 5 members
• Role: Front-End and ML Developer
Blind’s Aid HTML/CSS, JavaScript, Python, Django, NLP, Speech Recognition Nov 22 - Feb 23
• Developed a Voice-Based Email System for visually challenged users, enabling email management through voice
commands for all Google-verified accounts
• Team Project: Collaborated with a team of 5 members
• Role: Front-End and ML Developer
The Blood Map Project HTML/CSS, JavaScript, Python, NLP, Speech Recognition Dec 23 - May 24
• Developed a platform to efficiently connect hospitals and blood banks with nearby donors, ensuring timely blood supply
during emergencies by quickly locating and notifying eligible donors.
• Team Project: Collaborated with a team of 7 members
• Role: Integrated Google Translate API for seamless multilingual support, improving user accessibility.
• Authored comprehensive project documentation, ensuring organized and easily accessible information for future development
and maintenance
Certifications
• Applied Machine Learning in Python by University of Michigan - Coursera
• Java Certified Foundations Associate - Oracle
• Java Development on Oracle Cloud - Oracle
• Privacy and Security in Online Social Media by Prof. Ponnurangam Kumaraguru - NPTEL
• Introduction to Cybersecurity - CISCO
• Computer Vision - VITyarthi
Technical Skills
• Primary: Java, Python, HTML/CSS, JavaScript, MySQL
• Developer Tools: Git, GitHub, Visual Studio Code
Co-Curriculars
Member of the IoT Club 2023 - 2024
• Collaborated with the IoT Club’s social media team to organize events and boost engagement
• Developed and implemented social media strategies, resulting in increased account interactions and higher participation in the
club’s initiatives, enhancing overall visibility and outreach
• Languages: English, Hindi, Telugu, Sanskrit, Spanish
• Hobbies: Trained Basketball player, Trained Kuchipudi dancer, Love gardening and photography

"""
predicted_category= job_recommendation(resume_file)
print("Predicted Category:", predicted_category)

Predicted Category: Backend Developer


In [14]:
import pickle
pickle.dump(rf_classifier,open('rf_classifier_job_recommendation.pkl','wb'))
pickle.dump(tfidf_vectorizer,open('tfidf_vectorizor_job_recommendation.pkl','wb'))